In [2]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json


%matplotlib inline 

In [3]:
# Load .env enviroment variables
load_dotenv() 

True

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2 
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"  
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD" 